# atomman.lammps.normalize(system)

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-09-02

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The normalize function returns a new System identical to an existing one but with Box vectors and atomic positions transformed to be compatible with LAMMPS. All atom positions are adjusted to be within the box ([wrapped](atomman.System.wrap.ipynb)) and System.box's vectors transformed to be compatible. This function is called by the [System.normalize](atomman.System.normalize.ipynb) method with style='lammps'.

Arguments:

- __system__ -- a [System](atomman.System) to transform into a LAMMPS compatible System. 

In LAMMPS, the Box vectors must conform to the following rules:

    avect = [ lx, 0.0, 0.0]
    bvect = [ xy,  ly, 0.0]
    cvect = [ xz,  yz,  lz]
    avect, bvect, cvect are right-handed.
    
__WARNING!__ Tilt factors are not tested for compliance with LAMMPS. If large angles/tilt factors are present, the command "box tilt large" may be required in the LAMMPS input script.

The underlying code can be found in [atomman/lammps/normalize.py](../../atomman/lammps/normalize.py).

## Demonstration

In [1]:
from copy import deepcopy

#External library imports
import numpy as np

#atomman imports
import atomman as am
import atomman.lammps as lmp

### Normalize a messy random System

Create a messy test system with atoms outside bounds and random box vectors

In [2]:
atoms = am.Atoms(natoms=10, prop={'atype':1, 'pos':10*np.random.random((10,3))})
box = am.Box(vects=5*np.random.random((3,3)), origin=(5, 5, 5))

system = am.System(atoms=atoms, box=box)
print system

avect =  [ 1.033,  4.506,  0.678]
bvect =  [ 4.516,  1.386,  2.780]
cvect =  [ 2.828,  1.152,  4.189]
origin = [ 5.000,  5.000,  5.000]
natoms = 10
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.853 |   8.200 |   5.671
      1 |       1 |   4.785 |   4.057 |   9.536
      2 |       1 |   7.440 |   7.229 |   0.735
      3 |       1 |   1.097 |   5.371 |   9.550
      4 |       1 |   1.974 |   7.317 |   2.994
      5 |       1 |   8.009 |   2.150 |   7.080
      6 |       1 |   2.828 |   9.541 |   6.974
      7 |       1 |   0.345 |   1.261 |   4.798
      8 |       1 |   1.521 |   6.633 |   8.956
      9 |       1 |   6.968 |   9.607 |   0.617


Create a new normalized system

In [3]:
lammps_system = lmp.normalize(system)
print lammps_system

avect =  [ 4.673,  0.000,  0.000]
bvect =  [ 2.739,  4.748,  0.000]
cvect =  [-2.344, -4.127,  2.086]
origin = [ 0.000,  0.000,  0.000]
natoms = 10
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   3.056 |  -2.682 |   1.461
      1 |       1 |   5.164 |   3.591 |   0.183
      2 |       1 |   4.019 |   2.786 |   0.759
      3 |       1 |   1.340 |   0.834 |   0.342
      4 |       1 |   4.014 |   0.637 |   0.150
      5 |       1 |   0.547 |   0.148 |   1.859
      6 |       1 |   0.303 |  -0.756 |   1.393
      7 |       1 |   0.798 |  -1.705 |   1.880
      8 |       1 |   2.565 |   0.552 |   1.081
      9 |       1 |   1.519 |   1.709 |   0.633


### Check that the cell parameters of the original and normalized systems match

Lattice parameters match, and lattice angles will either match or old+new values add up to 180.

In [4]:
print 'a =',     system.box.a
print 'b =',     system.box.b
print 'c =',     system.box.c
print 'alpha =', system.box.alpha
print 'beta =',  system.box.beta
print 'gamma =', system.box.gamma

a = 4.6726922553
b = 5.48140547354
c = 5.18405090082
alpha = 23.7288916622
beta = 63.119322658
gamma = 60.0218920281


In [5]:
print 'a =',     lammps_system.box.a
print 'b =',     lammps_system.box.b
print 'c =',     lammps_system.box.c
print 'alpha =', lammps_system.box.alpha
print 'beta =',  lammps_system.box.beta
print 'gamma =', lammps_system.box.gamma

a = 4.6726922553
b = 5.48140547354
c = 5.18405090082
alpha = 156.271108338
beta = 116.880677342
gamma = 60.0218920281


### Check that relative atom positions are consistent and in the Box

The original System has atoms outide the System's Box (scaled positions not between 0 and 1)

In [6]:
print system.atoms_prop(key='pos', scale=True)

[[ 0.97968864 -1.95621701  1.29969984]
 [-0.33885808 -1.16757256  1.91238222]
 [ 0.51333952  1.90306254 -2.36390668]
 [ 0.182523   -2.68159359  2.83584829]
 [ 0.77971634 -0.80318248 -0.07205262]
 [-0.90824471  0.80605839  0.10855308]
 [ 1.15294418 -1.57867676  1.332045  ]
 [-0.62582747 -1.57545991  1.09848724]
 [ 0.47661908 -2.43330318  2.48175593]
 [ 1.11166405  1.62358257 -2.30331483]]


Wrapping the system corrects this

In [7]:
system.wrap()
print system.atoms_prop(key='pos', scale=True)

[[ 0.97968864  0.04378299  0.29969984]
 [ 0.66114192  0.83242744  0.91238222]
 [ 0.51333952  0.90306254  0.63609332]
 [ 0.182523    0.31840641  0.83584829]
 [ 0.77971634  0.19681752  0.92794738]
 [ 0.09175529  0.80605839  0.10855308]
 [ 0.15294418  0.42132324  0.332045  ]
 [ 0.37417253  0.42454009  0.09848724]
 [ 0.47661908  0.56669682  0.48175593]
 [ 0.11166405  0.62358257  0.69668517]]


The lammps normalized System has atomic positions consistent with the wrapped System. The relative z coordinates may be flipped if the original box vectors were left-handed.

In [8]:
print lammps_system.atoms_prop(key='pos', scale=True)

[[ 0.97968864  0.04378299  0.70030016]
 [ 0.66114192  0.83242744  0.08761778]
 [ 0.51333952  0.90306254  0.36390668]
 [ 0.182523    0.31840641  0.16415171]
 [ 0.77971634  0.19681752  0.07205262]
 [ 0.09175529  0.80605839  0.89144692]
 [ 0.15294418  0.42132324  0.667955  ]
 [ 0.37417253  0.42454009  0.90151276]
 [ 0.47661908  0.56669682  0.51824407]
 [ 0.11166405  0.62358257  0.30331483]]


- - -
__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)